<a href="https://colab.research.google.com/github/jimmiereevekim/Bok-Choi-Growth-Prediction-Model/blob/main/Bok_Choi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import random
import pandas as pd
import numpy as np
import os
import glob

import cv2
from time import time

import warnings
warnings.filterwarnings(action='ignore') 

import xgboost
from xgboost import plot_importance

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler 

scaler_m = MinMaxScaler()

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
import matplotlib.pyplot as plt
import statistics # 중앙값 

from collections import Counter 

import seaborn as sns
from scipy import stats


In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
#train input / target 와 test train / target을 각각 리스트에 전부 저장
all_input_list = sorted(glob.glob('/content/gdrive/MyDrive/open/train_input/*.csv'))
all_target_list = sorted(glob.glob('/content/gdrive/MyDrive/open/train_target/*.csv'))

train_input_list = all_input_list
train_target_list = all_target_list

test_input_list = sorted(glob.glob('/content/gdrive/MyDrive/open/test_input/*.csv'))
test_target_list = sorted(glob.glob('/content/gdrive/MyDrive/open/test_target/*.csv'))

In [14]:
# 이상치 처리 함수 정의 - IQR방식 채택
def IQR(df) :
  q3 = df.quantile(0.75)
  q1 = df.quantile(0.25)
  med = statistics.median(df)
  iqr = q3 - q1
  df = df.where(df < (q3 +1.5 *iqr), med)
  df = df.where(df > (q1 -1.5 *iqr ),med)
  return df


In [15]:
#타겟 전체 데이터를 읽은 후에 하나의 데이터프레임으로 묶음
#57번 파일은 input data에 결측치가 많아서 버림
df = pd.DataFrame() 
len_target = []

for i in train_target_list :
  if i == train_target_list[56]:
    continue
  target = pd.read_csv(i) 
  len_target.append(len(target))  
  df = pd.concat([df, target['rate']], ignore_index = True)

mean_data = pd.DataFrame()




In [18]:
# input 파일마다 parameter의 이름이 달라서 첫번째 파일의 이름으로 통일 
first_df = pd.read_csv(train_input_list[0])
cols = first_df.columns

#minute 담을 리스트
minute_l = []
case = 1
for i in train_input_list :
  #57번 파일은 스킵
  if i == train_input_list[56]:
   continue
  train = pd.read_csv(i)
  #42번 파일은 초기 787개의 데이터가 의미가 없어서 버림
  if  i == train_input_list[42]:
    train = train.iloc[787:]
  train.columns = cols
  #input데이터의 결측치를 채워줌
  train = train.ffill()
  train = train.bfill()
  #시계열 데이터에서 시간 자체는 학습에 영향을 주지 않으므로 버린다
  train = train.drop(columns = '시간')
  feature_importance = train.columns

  print('{}th file processing..'.format(case), end = ' ')
  
  #input 데이터가 1분 단위로 주어져 있으므로, 하루 단위로 이상치 처리 및 평균을 내주어야한다
  l = int(len(train)) // 1440
  j = 1 
  while j <= l :
    minute = train.iloc[1440*(j-1): 1440 * j] # 데이터프레임으로 하루 단위 추출 
    for num in feature_importance:
      #상태와 남은시간 관련 parameter들은 켜졌다 꺼졋다 등의 이분적인 데이터라 결측치처리후에 데이터가 망가질까봐 그 외의 데이터만 이상치를 처리한다.
      if ('상태' or '남은시간') not in num:
         minute[num] = IQR(minute[num]) # IQR사용해서 이상치 제거함
    minute_l.append(minute)
    j += 1
  print('Done!')
  case += 1


1th file processing.. Done!
2th file processing.. Done!
3th file processing.. Done!
4th file processing.. Done!
5th file processing.. Done!
6th file processing.. Done!
7th file processing.. Done!
8th file processing.. Done!
9th file processing.. Done!
10th file processing.. Done!
11th file processing.. Done!
12th file processing.. Done!
13th file processing.. Done!
14th file processing.. Done!
15th file processing.. Done!
16th file processing.. Done!
17th file processing.. Done!
18th file processing.. Done!
19th file processing.. Done!
20th file processing.. Done!
21th file processing.. Done!
22th file processing.. Done!
23th file processing.. Done!
24th file processing.. Done!
25th file processing.. Done!
26th file processing.. Done!
27th file processing.. Done!
28th file processing.. Done!
29th file processing.. Done!
30th file processing.. Done!
31th file processing.. Done!
32th file processing.. Done!
33th file processing.. Done!
34th file processing.. Done!
35th file processing.. 

In [19]:
#일별로 나눈 58개의 청경채를 하나의 데이터프레임으로 묶음
mean_data = pd.concat(minute_l)

In [20]:
#묶은 데이터에서 하루 단위로 평균을 냄
temp = pd.DataFrame()
l = int(len(mean_data)) // 1440
j = 1 
while j <= l :
  minute = mean_data.iloc[1440*(j-1): 1440 * j]
  minute = pd.DataFrame(minute.mean())
  minute = minute.transpose()
  temp = pd.concat([temp,minute], ignore_index = False)
  j += 1

In [21]:
mean_data = temp 

In [22]:
#데이터 스케일 조정 - minmaxscaler활용
mean_data = pd.DataFrame(mean_data)
#parameter와 성장률의 correlation을 확인하기 위해 데이터프레임에 정답 열을 추가
mean_data['target'] = list(df[0])
mean_cols = mean_data.columns
mean_data = scaler_m.fit_transform(mean_data)
mean_data = pd.DataFrame(mean_data, columns = mean_cols)
print(mean_data.head())

    내부온도관측치   내부습도관측치    CO2관측치     EC관측치   외부온도관측치   외부습도관측치      펌프상태  \
0  0.671538  0.310697  0.202174  0.000258  0.217460  0.110606  0.005650   
1  0.653485  0.352866  0.166706  0.000256  0.201606  0.122458  0.022607   
2  0.652416  0.354104  0.160439  0.000256  0.206725  0.159562  0.018079   
3  0.682608  0.289748  0.151924  0.000257  0.228072  0.225574  0.019217   
4  0.700641  0.454641  0.184712  0.000257  0.251130  0.245699  0.019209   

   펌프작동남은시간     최근분무량   일간누적분무량  ...      냉방온도      난방온도      기준온도  난방부하  \
0       0.0  0.000616  0.000081  ...  0.799892  0.815879  0.839098   0.0   
1       0.0  0.002179  0.005517  ...  0.799851  0.815833  0.839052   0.0   
2       0.0  0.002178  0.004821  ...  0.799851  0.815833  0.839052   0.0   
3       0.0  0.002176  0.005130  ...  0.799851  0.815834  0.839052   0.0   
4       0.0  0.002183  0.004882  ...  0.799851  0.815833  0.839052   0.0   

       냉방부하  총추정광량  백색광추정광량  적색광추정광량  청색광추정광량    target  
0  0.006056    0.0      0.0      0

In [23]:
#상관관계 확인
df_corr = mean_data.corr()['target']
df_corr = pd.DataFrame(df_corr.sort_values(ascending=False), columns=['target'])
df_corr.style.background_gradient(cmap='viridis')


,target
target,1.000000
총추정광량,0.081927
백색광추정광량,0.071274
블루 LED동작강도,0.066225
청색광추정광량,0.063063
화이트 LED동작강도,0.054017
CO2관측치,0.047068
적색광추정광량,0.039136
냉방작동남은시간,0.030338
레드 LED동작강도,0.017301


In [24]:
#상관관계는 0.75아상인 데이터가 적합하다고 판단하여 그 데이터만 feature로 가져감
df_corr[abs(df_corr['target'])>=0.075].index


Index(['target', '총추정광량', '내부습도관측치'], dtype='object')

In [25]:
feature = ['백색광추정광량', 'EC관측치', '총추정광량', '화이트 LED동작강도', '기준온도', '난방온도',
       'CO2관측치', '냉방작동남은시간', '펌프작동남은시간', '외부환기팬작동남은시간', '난방부하', '외부습도관측치',
       '내부습도관측치']
mean_data = mean_data[feature]

In [26]:
#학습에 필요한 파라미터 정리
X = pd.DataFrame(mean_data)
y = df

In [27]:
#kfold와 xgboost모델을 활용하여 모델 학습
from sklearn.model_selection import KFold

In [28]:
l_model = []
k_fold = KFold(n_splits = 5, shuffle=True, random_state = 123)
for train_idx, val_idx in k_fold.split(X) :
  model = xgboost.XGBRegressor(
     objective = 'reg:squarederror',
     n_estimators = 5000,
     learning_rate = 0.005, 
     num_boost_around = 1000,
     max_depth = 6,
     eta = 0.008,
     sum_sample = 0.9
    ) 

  x_t = X.iloc[train_idx]
  y_t = y.iloc[train_idx]
  x_val = X.iloc[val_idx]
  y_val = y.iloc[val_idx]

  l_model.append(model.fit(x_t,y_t, eval_set= [(x_val,y_val)], early_stopping_rounds = 100))

[0]	validation_0-rmse:0.393166
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:0.392329
[2]	validation_0-rmse:0.391522
[3]	validation_0-rmse:0.390732
[4]	validation_0-rmse:0.389984
[5]	validation_0-rmse:0.389253
[6]	validation_0-rmse:0.388564
[7]	validation_0-rmse:0.387892
[8]	validation_0-rmse:0.387259
[9]	validation_0-rmse:0.386657
[10]	validation_0-rmse:0.386069
[11]	validation_0-rmse:0.385539
[12]	validation_0-rmse:0.385008
[13]	validation_0-rmse:0.384529
[14]	validation_0-rmse:0.384095
[15]	validation_0-rmse:0.383674
[16]	validation_0-rmse:0.383296
[17]	validation_0-rmse:0.382499
[18]	validation_0-rmse:0.382235
[19]	validation_0-rmse:0.381917
[20]	validation_0-rmse:0.381713
[21]	validation_0-rmse:0.381557
[22]	validation_0-rmse:0.380846
[23]	validation_0-rmse:0.380606
[24]	validation_0-rmse:0.380513
[25]	validation_0-rmse:0.380454
[26]	validation_0-rmse:0.380276
[27]	validation_0-rmse:0.380258
[28]	validation_0-rmse:0.379656
[29]	validation_

In [30]:
#test데이터도 같은 방법으로 전처리 해줌
for case in range(6) :
  preds = []
  test = pd.read_csv(test_input_list[case])
  test.fillna(0)
  test.columns = cols 
  time = test[cols[0]]
  l = int(len(test)) // 1440
  j = 1 
  day = []

  while j <= l :
    minute = test.iloc[1440*(j-1): 1440 * j] 
    day.append(minute)
    j += 1 
  
  temp = pd.concat(day)
  temp = temp.ewm(span=10).mean()
  temp['시간'] = time  

  temp.dropna(subset=['시간'],inplace=True)
  temp['시간'] = pd.to_datetime(temp['시간'])
  temp['년월일'] = temp['시간'].dt.year.astype(str) + '-' + temp['시간'].dt.month.astype(str) + '-' + temp['시간'].dt.day.astype(str)
  temp['년월일'] = pd.to_datetime(temp['년월일'])
  temp_mean = temp.groupby('년월일').mean()
  temp_mean = temp_mean.reset_index(drop=True)

  for model in l_model :
    test_df = temp_mean[feature]
    preds.append(model.predict(test_df))
  
  #모델로 예측한 값으로 새로운 데이터 프레임을 만들고 정답 파일 만듬
  y_test = pd.read_csv(test_target_list[case]) # 정답
  ans = pd.DataFrame(preds)
  ans = ans.mean()
  submit_df = pd.read_csv(test_target_list[case])
  submit_df['rate'][0:] = ans[len(ans)-len(submit_df):]
  submit_df.to_csv(test_target_list[case],index = False)

In [31]:
#정답 파일을 zip파일로 만들고 드라이브에 올림
import zipfile
os.chdir("/content/gdrive/MyDrive/open/test_target/")
submission = zipfile.ZipFile("../submission.zip", 'w')
for path in test_target_list:
    path = path.split('/')[-1]
    submission.write(path)
submission.close()